# Exercise 2

Analyze the Kepler problem in cartesian coordinates

\begin{equation}
    L = \frac{1}{2}\mu (\dot{x}^2+\dot{y}^2+\dot{z}^2)
        +\frac{G\mu M}{\sqrt{x^2+y^2+z^2}}\ .
\end{equation}

Find the Euler-Lagrange equations for this Lagrangian. Rewrite them as a system of first-order equations (convenient for numerical solutions). Repeat the problem changing to spherical coordinates. Solve the system numerically and plot the orbit.

### Import sympy and define symbols

In [1]:
from sympy import *
init_printing()  # import the best printer available
M, mu, G, t = symbols('M mu G t', positive=True)
x, y, z = symbols('x y z', cls=Function)

### Define your lagrangian

In [ ]:
L = mu*(x(t).diff()**2 + y(t).diff()**2 + z(t).diff()**2)/2 \
    + G*mu*M/sqrt(x(t)**2 + y(t)**2 +z(t)**2)

L

### Find the Euler-Lagrange equations

In [ ]:
cart_vars = [x(t), y(t), z(t)]
system = [Eq( (L.diff(q.diff())).diff(t), L.diff(q) ) for q in cart_vars]

system

### Rewrite the Euler-Lagrange equations as a system of first order ODEs

We define the momenta and define a list with all the variables. **CAREFUL!**: respect the order of variables throughout the program, otherwise it may be a good idea to define the equations through a dictionary, with the variable as key.

In [ ]:
px, py, pz = symbols('p_x p_y p_z', cls=Function)

cart_momenta = [px(t), py(t), pz(t)]
full_cart_vars = cart_vars + cart_momenta

full_cart_vars

We must invert the velocities in terms of the momenta

In [ ]:
cart_vel = [q.diff(t) for q in cart_vars]

vel_momenta = [Eq(p, L.diff(dq)) for dq, p in zip(cart_vel, cart_momenta)]
vel_momenta

In [ ]:
sol = solve(vel_momenta, cart_vel)
sol

Define the first half of the system, just as the velocities in terms of momenta.

In [ ]:
system_1st = [Eq( dq, sol[dq]) for dq in cart_vel]
system_1st

The second half comes from the Euler-Lagrange equations

In [ ]:
temp = [Eq(p.diff(t), L.diff(q)) for p, q in zip(cart_momenta, cart_vars)]
temp

Define the total system joining both sets (beware of the order! must be the same as `full_cart_vars`).

In [ ]:
system_1st += temp
system_1st

### Change to spherical coordinates in the lagrangian and repeat the steps above

Define the substitution rule to change to spherical coordinates

In [ ]:
r, th, phi = symbols('r theta phi', cls=Function)

cart_to_polar = {x(t): r(t)*sin(th(t))*cos(phi(t)),
                 y(t): r(t)*sin(th(t))*sin(phi(t)),
                 z(t): r(t)*cos(th(t))}
cart_to_polar

and the lagrangian in the new coordinates

In [ ]:
L_polar = L.subs(cart_to_polar).doit().simplify()
L_polar

In this case, the Euler-Lagrange equations are a bit messy

In [ ]:
polar_vars = [r(t), th(t), phi(t)]
polar_vel = [q.diff(t) for q in polar_vars]

system_polar = [Eq( (L_polar.diff(q_dot)).diff(t) - L_polar.diff(q), 0)\
                for q, q_dot in zip(polar_vars, polar_vel)]

system_polar

We define the accelerations and solve for them

In [ ]:
accel = [q.diff(t, 2) for q in polar_vars]
accel

In [ ]:
sol = solve(system_polar, accel)
sol

Now we have something simpler

In [ ]:
system_polar = [Eq( ddq, sol[ddq].simplify() ) for ddq in accel]
system_polar

Again, we rewrite it as a system of first order differential equations

In [ ]:
pr, pth, pphi = symbols('p_r p_theta p_phi', cls=Function)

polar_momenta = [pr(t), pth(t), pphi(t)]
full_polar_vars = polar_vars + polar_momenta

full_polar_vars

In [ ]:
vel_momenta = [Eq(p, L_polar.diff(dq)) for p, dq in zip(polar_momenta, polar_vel)]
vel_momenta

In [ ]:
sol = solve(vel_momenta, polar_vel)
sol

In [ ]:
system_polar_1st = [Eq( dq, sol[dq]) for dq in polar_vel]
system_polar_1st

In [ ]:
temp = [Eq(p.diff(t), L_polar.diff(q).subs(sol).simplify()) \
        for p, q in zip(polar_momenta, polar_vars)]
temp

In [ ]:
system_polar_1st += temp
system_polar_1st

### Solve numerically the equations and plot the orbit

I will solve only the simplified case $p_\theta = 0$ and $\theta=\pi/2$ (we can achieve this re-orienting the axes)

In [ ]:
new_system = [var.subs(pth(t), 0).subs(th(t), pi/2).doit() for var in system_polar_1st]
new_system

We will consider then a reduced set of variables and equations

In [ ]:
# delete trivial equations 
# -> careful we must use sympy's true instead of python's True
final_system = [eq for eq in new_system if eq is not true]
final_system

In [ ]:
# construct the new set of variables
final_vars = [eq.lhs.args[0] for eq in final_system]
final_vars

In [ ]:
import numpy as np
from scipy.integrate import odeint

# parameters, -> CAREFUL!  do not redefine symbolic variables here
nmu = 1.
nM = 10000.
nG = 1.

# initial conditions
r0 = 10.
phi0 = 0.
pr0 = 0.
pphi0 = 1.3*np.sqrt(nG*nM*r0)

y0 = [r0, phi0, pr0, pphi0]

# array of sampled times for the solution
nt = np.linspace(0, 50, 500)

Now we could define the numeric system as

In [ ]:
def num_system(y, t, mu, M, G):
    r, phi, pr, pphi = y
    
    dr = pr/mu
    dphi = pphi/mu/r/r
    dpr = -G*M*mu/r/r + pphi*pphi/mu/r/r/r
    dpphi = 0
    
    return dr, dth, dpr, dpth

But we will try a different approach, useful for heavy equations : let `sympy` do the hard work with `lambdify`

In [ ]:
total_vars = [final_vars, t, mu, M, G]
total_vars

Collect the right-hand side of the differential equations (we are mimicking the numeric function above)

In [ ]:
rhs_sym = [eq.rhs for eq in final_system]
rhs_sym

Create a numerical system equivalent to the one defined by hand (sometimes it is better to write the system by hand, to reorder terms or define intermediate variables, that may be repeated many times)

In [ ]:
num_system = lambdify(total_vars, rhs_sym, "numpy")

Solve everything as usual with `odeint`

In [ ]:
num_solution = odeint(num_system, y0, nt, args=(nmu, nM, nG)).transpose()

and plot the orbit

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

R, Phi, PR, PPhi = num_solution

X = R*np.cos(Phi)
Y = R*np.sin(Phi)

plt.figure(1)
plt.axes().set_aspect('equal')   # aspect ratio of the axes
plt.plot(0, 0, 'o', color='C1')
plt.plot(X, Y, color='C2');